In [ ]:
# import requests
# import pathlib
# import os
# import zipfile
# import re
# import pandas as pd

In [ ]:
from data_collection import download_cot_zip, aggregate_reports
import pathlib
from tqdm import tqdm
from datetime import datetime
import pandas as pd

### Configuration

In [ ]:
data_location = pathlib.Path('data/zip')

In [ ]:
overwrite = False
if (pathlib.Path('data/zip/deacot1988.zip').is_file()) & (not overwrite):
    print("SKIP FILE")
else:
    print("DOWNLOAD")

#### Download all futures only data from www.cftc.gov

In [ ]:
for year in tqdm(range(1986, datetime.now().year + 1)):
    download_cot_zip(year, data_location)

### Parse to single parquet

In [ ]:
df = pd.DataFrame()
for file_path in tqdm(data_location.glob('*.zip')):
    df = pd.concat([df, parse_annual_zip(file_path)])
df.sort_values(by=['product', 'report_date'], ascending=True)

In [ ]:
t = df.sort_values(by=['product', 'report_date'], ascending=True)

In [ ]:
t.loc[('E-MINI S&P 500 STOCK INDEX - CHICAGO MERCANTILE EXCHANGE', slice(None))]

In [ ]:
t.to_parquet('data/all_cot.parquet')

In [ ]:
features_list = df.index.get_level_values(0).unique()
latest_report_date = t.index.get_level_values(1).max()

In [ ]:
features_list[features_list.str.contains("S&P")]

In [ ]:
pd.concat([df,df2])

In [ ]:
pd.read_csv("data/zip/deacot1989.zip").columns

In [ ]:
df = df.rename(columns={
    'Market and Exchange Names': 'product',
    'As of Date in Form YYMMDD': 'report_date',
    'Noncommercial Positions-Long (All)': 'noncommercial_long',
    'Noncommercial Positions-Short (All)': 'noncommercial_short',
    'Commercial Positions-Long (All)': 'commercial_long',
    'Commercial Positions-Short (All)': 'commercial_short',
}).set_index(['product', 'report_date'])

In [ ]:
df.info()

### OLD STUFF

In [ ]:
data_folder = 'data'
year = 2017
file_path = pathlib.Path(data_folder, f"{year}.csv")
zip_path = pathlib.Path(f"{data_folder}/zip/deacot{year}.zip")
zip_path

In [ ]:
df = pd.read_csv(zip_path, engine="pyarrow")
df.info()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
def list_files(filepath: str, filetype: str) -> list[str]:
    paths = []
    for root, dirs, files in os.walk(filepath):
        for file in files:
            if file.lower().endswith(filetype.lower()):
                paths.append(os.path.join(root, file))

                
def unzip_file(zip_path: str, extract_name: str, extract_folder: str): 
    with zipfile.ZipFile(zip_path, "r") as zf:
        zipinfos = zf.infolist()

        for zipinfo in zipinfos:
            zipinfo.filename = extract_name
            zf.extract(zipinfo, extract_folder)

In [ ]:
url = f'https://www.cftc.gov/files/dea/history/deacot{year}.zip'
r = requests.get(url)
with open(f'data/zip/deacot{year}.zip', 'wb') as outfile:
    outfile.write(r.content)

In [ ]:
def unzip_file(zip_path: str, extract_name: str, extract_folder: str): 
    with zipfile.ZipFile(zip_path, "r") as zf:
        zipinfos = zf.infolist()

        for zipinfo in zipinfos:
            zipinfo.filename = extract_name
            zf.extract(zipinfo, extract_folder)

In [ ]:
extract_folder = 'data'
files = list_files(filepath='data/zip', filetype='zip')
for file in files:
    print(file)
    zips = re.search('(\d{4}_\d{4})|(\d{4})', file)
    if zips:
        extract_name = f"futures_only_{zips[0]}.csv"
        unzip_file(zip_path=file, extract_name=extract_name, extract_folder=extract_folder)